In [2]:
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sb

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.functions import corr
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import DecisionTreeClassificationModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import GBTClassificationModel

In [3]:
sc = SparkContext(master="local", appName="New Spark Context")
spark = SparkSession(sc)

Part 1: Chapter 6

In [12]:
# df = spark.read.csv("./data/Ecommerce_Customers.csv", header=True, inferSchema=True)
# df.show(5)

In [16]:
# df.printSchema()

In [17]:
# df.columns

In [15]:
input_cols = df.columns[3:-1]
target_col = df.columns[-1]

assembler = VectorAssembler(inputCols=input_cols, outputCol='features')
df_pre = assembler.transform(df)
final_df = df_pre.select('features', 'Yearly Amount Spent')

NameError: name 'df' is not defined

In [ ]:
df_pre[['features']].show(5, False)

In [ ]:
train_df, test_df = final_df.randomSplit([0.7, 0.3], seed=42)
train_df.describe().show()

In [ ]:
test_df.describe().show()

In [15]:
lr = LinearRegression(featuresCol='features', 
                      labelCol='Yearly Amount Spent',
                      predictionCol='Predict_Yearly_Amount_Spent'
                      )
model = lr.fit(train_df)

In [17]:
print('Coefficients: {} -  Intercept: {}'.format(model.coefficients, model.intercept))

Coefficients: [25.93215475417773,38.31861228860246,0.6756981758947298,61.36799727890117] -  Intercept: -1061.6126556938295


In [21]:
test_results = model.evaluate(test_df)
test_results.residuals.show(5)

+-------------------+
|          residuals|
+-------------------+
| 0.5230324245964653|
|-17.069653184407684|
|  7.546772405139109|
|-20.438521626093575|
| 23.359389939390212|
+-------------------+
only showing top 5 rows



In [22]:
test_results = model.evaluate(test_df)

print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))
print('MAE: {}'.format(test_results.meanAbsoluteError))
print('R2: {}'.format(test_results.r2))

RMSE: 9.850957450983932
MSE: 97.04136270109586
MAE: 7.933457771534967
R2: 0.9849403386897637


In [23]:
predictions = model.transform(test_df)
predictions.columns

['features', 'Yearly Amount Spent', 'Predict_Yearly_Amount_Spent']

In [24]:
predictions[['Yearly Amount Spent', 'Predict_Yearly_Amount_Spent']].show(5)

+-------------------+---------------------------+
|Yearly Amount Spent|Predict_Yearly_Amount_Spent|
+-------------------+---------------------------+
| 442.06441375806565|          441.5413813334692|
|   266.086340948469|          283.1559941328767|
|  494.6386097568927|          487.0918373517536|
|  486.9470538397658|         507.38557546585935|
|  591.7810894256675|          568.4216994862772|
+-------------------+---------------------------+
only showing top 5 rows



In [25]:
model.save('linear_model_ecommerce_customers')

In [28]:
from pyspark.ml.regression import LinearRegressionModel
model2 = LinearRegressionModel.load('linear_model_ecommerce_customers')

In [29]:
model2.evaluate(test_df).r2

0.9849403386897637

Part 2: Chapter 7

In [18]:
df = spark.read.csv("./data/flights.csv", header=True, inferSchema=True)
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [19]:
assembler = VectorAssembler(inputCols=['mile'], outputCol='features')
df_pre = assembler.transform(df)
final_df = df_pre.select('features', 'duration')

In [20]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)
train_df.describe().show()

+-------+------------------+
|summary|          duration|
+-------+------------------+
|  count|             39948|
|   mean|151.35360969260037|
| stddev| 86.74354736650423|
|    min|                30|
|    max|               560|
+-------+------------------+



In [21]:
test_df.describe().show()

+-------+------------------+
|summary|          duration|
+-------+------------------+
|  count|             10052|
|   mean|153.40399920413847|
| stddev| 88.21851423611517|
|    min|                31|
|    max|               560|
+-------+------------------+



In [22]:
lr = LinearRegression(featuresCol='features', 
                      labelCol='duration',
                      )
model = lr.fit(train_df)

In [23]:
print('Coefficients: {} -  Intercept: {}'.format(model.coefficients, model.intercept))

Coefficients: [0.12156049554670376] -  Intercept: 44.47745647046576


In [24]:
test_results = model.evaluate(test_df)

print('RMSE: {}'.format(test_results.rootMeanSquaredError))
print('MSE: {}'.format(test_results.meanSquaredError))
print('MAE: {}'.format(test_results.meanAbsoluteError))
print('R2: {}'.format(test_results.r2))

RMSE: 17.012577756944967
MSE: 289.4278019360987
MAE: 13.060488079800043
R2: 0.9628067632188161


Part 3: Pipeline

In [25]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

df = spark.read.csv("./data/flights.csv", header=True, inferSchema=True)
df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 5 rows



In [26]:
final_df = df.withColumn('km', round(df.mile * 1.60934, 0))
final_df = final_df.withColumn('label', (df.delay >= 15).cast('integer'))
final_df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| null|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|   419|ORD|1236| 10.33|     195|   -5|1989.0|    0|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA| 415.0| null|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 5 rows



In [27]:
no_label_df = final_df[final_df.label.isNull()]
no_label_df.show(5)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| null|
|  4|  2|  5|     AA|   325|ORD| 258|  8.92|      65|   NA| 415.0| null|
|  0|  8|  2|     UA|   549|ORD| 334| 11.08|      85|   NA| 538.0| null|
|  5|  8|  0|     UA|   574|ORD| 235| 14.48|      79|   NA| 378.0| null|
|  1| 13|  3|     US|  2189|LGA| 214|  20.0|      82|   NA| 344.0| null|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 5 rows



In [28]:
final_df = final_df[final_df.label.isNotNull()]
final_df = final_df.dropna()
final_df.count()

47022

In [29]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

In [30]:
indexer_carrier = StringIndexer(inputCol='carrier', outputCol='carrier_idx')
indexer_org = StringIndexer(inputCol='org', outputCol='org_idx')


onehot = OneHotEncoder(inputCols=['carrier_idx', 'org_idx', 'dow'],
                       outputCols=['carrier_dummy', 'org_dummy', 'dow_dummy'])

input_cols = ['km', 'carrier_dummy', 'org_dummy', 'dow_dummy']
assembler = VectorAssembler(inputCols=input_cols, outputCol='features')

lr = LogisticRegression()

pipeline = Pipeline(stages=[indexer_carrier, indexer_org, onehot, assembler, lr])

In [31]:
model = pipeline.fit(train_df)
predictions = model.transform(test_df)
predictions = predictions.withColumn('label', predictions.label.cast(DoubleType()))
predictions.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  1.0|       1.0|
|  1.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  1.0|       1.0|
+-----+----------+
only showing top 5 rows



In [32]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
prediction_and_label = predictions.select(['prediction', 'label'])
metrics = MulticlassMetrics(prediction_and_label.rdd)

print('Accuracy: {:.4f}'.format(metrics.accuracy))
print('Precisions: {:.4f}'.format(metrics.precision(label=1)))
print('Recall: {:.4f}'.format(metrics.recall(label=1)))
# print('f1_score: {:.4f}'.format(metrics.fMeasure(label=1)))

Accuracy: 0.5906
Precisions: 0.5805
Recall: 0.7440


* Đánh giá kết quả

In [36]:
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(predictions, {
    multi_evaluator.metricName: "weightedPrecision"
})

In [37]:
weighted_precision

0.5949207645746231

In [40]:
binary_evaluator = BinaryClassificationEvaluator()

In [42]:
auc = binary_evaluator.evaluate(predictions, {
    binary_evaluator.metricName: "areaUnderROC"
})

In [43]:
auc

0.6135788258477005

* Lưu model và load

In [68]:
path = r'./data/logiticModel_flights'

In [69]:
model.save(path)

In [70]:
from pyspark.ml import PipelineModel

In [71]:
logisticModel2 = PipelineModel.load(path)

In [ ]:
def smthing()